In [1]:
import pandas as pd
import numpy as np
import requests
from pandas import DataFrame

In [2]:
df1= pd.read_csv('ads_content_7224.csv')
df2=pd.read_csv('socialbeta_all.csv')

In [3]:
content_ls=df1['ad_content'].to_list()
_=df2['content'].to_list()
content_ls +=_
df_content=pd.DataFrame(content_ls,columns=['content'])
df_content['content'].astype('str')
df_content.dropna(axis=0, how='any', inplace=True)

In [4]:
df_content.shape

(8387, 1)

In [5]:
import re
import math
import jieba
import os

In [6]:
# 对文章进行分词
#载入自定义的词典
jieba.load_userdict('/Users/julia/learndata/dic_jieba/jiebaDict.txt')

df_content['content'] = df_content['content'].apply(jieba.lcut)
#过滤停用词\长度小于1的词\非中文词
pattern = re.compile(u'[\u4e00-\u9fa5]+')
stopwords = [line.strip().encode('utf-8').decode('utf-8') for line in open('/Users/julia/learndata/dic_jieba/cn_stopwords.txt').readlines()]
# 判断字符串是不是纯中文
df_content['content']= df_content['content'].apply(
    lambda cut_words: [word for word in cut_words if word not in stopwords and len(word) > 1 and pattern.search(word)])

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/fy/71rk0bjs7rv6r71b4g21lqcc0000gn/T/jieba.cache
Loading model cost 0.741 seconds.
Prefix dict has been built successfully.


In [7]:
# 对每篇文章结果进行去重,并获取所有的分词结果
content_cut = df_content['content'].apply(lambda s: list(set(s)))
content_word=[]
for i in content_cut.index:
    content_word += content_cut[i]
content_word = set(content_word)


#构造字典，用于存放包含某个词汇文档的数量
path_num = os.path.join(r"/Users/julia/Desktop/", 'num_dict.txt')

try:
    # 先读取上次结果， 若不存在则重新构造
    dict_df = pd.read_csv(path_num, encoding='gbk', sep=' ', header=None, names=['word', 'idf'])
    dic = dict(zip(dict_df['word'], dict_df['idf']))
except Exception as e:
	# 新构造的字典初始值全为0
    dic = dict(zip(content_word, [0] * len(content_word)))

# 计算IDF值
# 1.更新出现每个词文章的数目
for i in content_cut.index:
    for word in content_cut[i]:
        try:
            dic[word] += 1
        except:
            dic[word] = 1
# 2.保存次数，用于下一次的增量更新
file = open(path_num, 'w', encoding='utf-8')
for key, value in dic.items():
    try:
        text = key + ' ' + str(value) + '\n'
        file.write(text)
    except Exception as e:
        print(e)
file.close()

# 3.计算IDF值
path_idf = os.path.join(r"/Users/julia/Desktop/", "idf_dict.txt")
n = df_content.shape[0]  # 文章总数
idf_dic = {key: math.log(n / value, 10) for key, value in dic.items()}
 # 保存，添加到结巴的提取关键词词库
file = open(path_idf, 'w', encoding='utf-8')
for key, value in idf_dic.items():
    try:
        text = key + ' ' + str(value) + '\n'
        file.write(text)
    except Exception as e:
        print(e)
file.close()